# Updating individual HAWC references

This notebook demonstrates using the HAWC client to update the fields of and delete individual HAWC references.

Make sure the `hawc_client` is installed. If you need to install it:

```bash
pip install -U hawc_client
```

Then, we'll create a new import, find our reference instance from that import, make changes to that reference, and then delete that reference, demonstrating these core functionalities. You must have write permissions for this assessment to perform these actions.

In [1]:
from getpass import getpass

from hawc_client import HawcClient

First, setup a HAWC client instance and authenticate with your username and password:

In [2]:
client = HawcClient("https://hawcproject.org")
client.authenticate(email="admin@hawcproject.org", password=getpass())
assessment_id = 100500159

 ········


Next, we will create a new HERO import and find our reference through the imported HERO ID. We will then use that reference for updating and deleting operations.

In [3]:
hero_id = 1037843

response = client.lit.import_hero(
    assessment_id, title="my import", description="import description", ids=[hero_id]
)

### Viewing our new reference

In [4]:
# get HAWC reference ID mapping
references = client.lit.reference_ids(assessment_id)
reference_id = int(references.query(f"`hero_id` == {hero_id}").reference_id.iloc[0])
reference_id

100524942

In [5]:
client.lit.reference(reference_id)

{'id': 100524942,
 'title': 'Application of High-speed Countercurrent Chromatography-Evaporative Light Scattering Detection for the Separation of Seven Steroidal Saponins from Dioscorea villosa',
 'authors_short': 'Yoon KD et al.',
 'authors': 'Yoon KD, Chin YW, Yang MH, Choi J, Kim J',
 'year': None,
 'journal': 'Phytochemical Analysis.',
 'abstract': 'INTRODUCTION: Steroidal saponins in Dioscorea species are chemically characterised as spirostanol and furostanol saponins, and have been used as standard marker compounds due to their chemotaxonomical significance and their important biological activities. OBJECTIVE: To design a simple, rapid and efficient method for the separation of steroidal saponins with a high degree of purity using high-speed countercurrent chromatography (HSCCC) coupled with evaporative light scattering detection (ELSD). METHODOLOGY: In the first step, reversed-phase mode HSCCC (flow rate: 1.5\u2009mL/min; revolution speed: 800\u2009rpm) using n-hexane:n-butanol:

## Updating references

With a reference ID and named parameters, a reference's fields can be updated. To update reference tags, a parameter `tags` should be passed with a list of tag IDs.

In [6]:
response = client.lit.update_reference(
    reference_id,
    title="new title",
    authors_short="new authors short",
    authors="new authors long",
    year=2020,
    journal="journal",
    abstract="abstract",
    full_text_url="https://google.com",
)
response

{'id': 100524942,
 'title': 'new title',
 'authors_short': 'new authors short',
 'authors': 'new authors long',
 'year': 2020,
 'journal': 'journal',
 'abstract': 'abstract',
 'full_text_url': 'https://google.com',
 'created': '2020-07-27T18:48:12.728175-05:00',
 'last_updated': '2020-07-27T18:48:13.196777-05:00',
 'block_id': None,
 'assessment': 100500159,
 'searches': [100500415],
 'identifiers': [100526410]}

You can even apply tags. Note that applying tags in this fashion will remove any tags previously applied to a reference and replace with the new set.  

In this example, we'll apply 3 random tags:

In [7]:
tags = client.lit.tags(assessment_id)
tags.head()

,id,depth,name,nested_name
0,100504028,2,Inclusion,Inclusion
1,100504029,3,Human Study,Inclusion|Human Study
2,100504030,3,Animal Study,Inclusion|Animal Study
3,100504031,3,Mechanistic Study,Inclusion|Mechanistic Study
4,100504032,2,Exclusion,Exclusion


In [8]:
response = client.lit.update_reference(reference_id, tags=tags.id.sample(3, replace=False).tolist())
response

{'id': 100524942,
 'title': 'new title',
 'authors_short': 'new authors short',
 'authors': 'new authors long',
 'year': 2020,
 'journal': 'journal',
 'abstract': 'abstract',
 'full_text_url': 'https://google.com',
 'created': '2020-07-27T18:48:12.728175-05:00',
 'last_updated': '2020-07-27T18:48:13.651524-05:00',
 'block_id': None,
 'assessment': 100500159,
 'searches': [100500415],
 'identifiers': [100526410]}

Confirm the tags were applied:

In [9]:
ref_tags = client.lit.reference_tags(assessment_id)
ref_tags.query(f"reference_id == {reference_id}").tag_id.to_list()

[100504032, 100504033, 100504034]

All of the HERO associated references in an assessment can also have their fields updated with the most current metadata pulled from HERO.

In [10]:
client.lit.update_references_from_hero(assessment_id)
client.lit.reference(reference_id)

{'id': 100524942,
 'title': 'Application of High-speed Countercurrent Chromatography-Evaporative Light Scattering Detection for the Separation of Seven Steroidal Saponins from Dioscorea villosa',
 'authors_short': 'Yoon KD et al.',
 'authors': 'Yoon KD, Chin YW, Yang MH, Choi J, Kim J',
 'year': None,
 'journal': 'Phytochemical Analysis.',
 'abstract': 'INTRODUCTION: Steroidal saponins in Dioscorea species are chemically characterised as spirostanol and furostanol saponins, and have been used as standard marker compounds due to their chemotaxonomical significance and their important biological activities. OBJECTIVE: To design a simple, rapid and efficient method for the separation of steroidal saponins with a high degree of purity using high-speed countercurrent chromatography (HSCCC) coupled with evaporative light scattering detection (ELSD). METHODOLOGY: In the first step, reversed-phase mode HSCCC (flow rate: 1.5\u2009mL/min; revolution speed: 800\u2009rpm) using n-hexane:n-butanol:

References can also have their HERO IDs replaces. This will also result in the fields being updated with the new HERO ID's metadata.

For this endpoint, all of the references must belong to the same assessment and this operation must not result in references with the same HERO ID.

In [11]:
new_hero = 1037655
replace = [[reference_id, new_hero]]
client.lit.replace_hero(assessment_id, replace)
client.lit.reference(reference_id)

{'id': 100524942,
 'title': 'Controlling Size, Amount, and Crystalline Structure of Nanoparticles Deposited on Graphenes for Highly Efficient Energy Conversion and Storage',
 'authors_short': 'Choi BG and Park HS',
 'authors': 'Choi BG, Park HS',
 'year': None,
 'journal': 'ChemSusChem.',
 'abstract': 'A facilitated electrochemical reaction at the surface of electrodes is crucial for highly efficient energy conversion and storage. Herein, various nanoparticles (NPs) including Au, Pt, Pd, Ru, and RuO(2) , were synthesized in\u2005situ and directly deposited on the ionic liquid (IL)-functionalized reduced graphene oxides (RGOs) in a controlled manner. The size, amount, and crystalline structures of discrete NPs were readily controlled, giving rise to enhanced methanol oxidation and pseudocapacitance. The well-defined nanostructure of decorated NPs and the favorable interaction between ILs and RGOs (or NPs) facilitated the electrochemical reaction, where NPs acted as electrocatalysts for 

## Deleting references

A reference can also be deleted from HAWC. 

**Warning:** This also removes the reference from any searches/imports which may have included the reference. If data was extracted with this reference and it is associated with bioassay or epi extractions they will also be removed.

In [12]:
client.lit.delete_reference(reference_id)